In [ ]:
import numpy as np
import pandas as pd
import math
import optuna
import os
import matplotlib.pyplot as plt
from matplotlib import rcParams
#import plotly.express as px
from optuna.visualization import plot_optimization_history, plot_slice
from optuna.samplers import TPESampler


In [ ]:
#This code block creates a runable abaqus model with a designed specimen using the kl_specimen module.
# Import the specimen design module
from specimen_design import create_kl_specimen

# Define your eigenvalues - this is an example list
eigenvalues =[78.827214,24.674865,36.650959,18.190706,28.634503,0.162061,12.249784,11.030737,6.482283,4.697120,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000]

# Generate ABAQUS script
abaqus_script = create_kl_specimen(
    eigenvalues=eigenvalues,
    target_void_fraction=0.25,
    corner_rounding=True,
    min_radius=0.5
)

In [ ]:
#Run the generated ABAQUS script
!abaqus cae noGUI=kl_specimen_abaqus.py

In [ ]:
#Evaluate stress state entropy using stress triaxaiality and Lode angle parameter as metrics

def cal_entropy(data_1):
    # Entropy
    #stress variables in in plane directions
    s11= data_1[:,0]    
    s22= data_1[:,1]
    s12= data_1[:,3]

    #Principal stresses
    S1=(s11+s22)/2 - ((((s11-s22)/2)**2)+s12**2)**0.5
    S2=(s11+s22)/2 + ((((s11-s22)/2)**2)+s12**2)**0.5

    #Ratio of the principal stresses Sig1/sig2
    S1_S2R=np.round(S1/S2)
    #print(S1/S2)
    #Equivalent stress
 
    F=0.278
    G=0.373
    H=0.627
    N=2.340
    #Sig_eq=((0.5*s22**2)+(0.5*s11**2)+(0.5*(s11-s22)**2)+(2*1.5*s12**2))**0.5
    Sig_eq=((G*s22**2)+(F*s11**2)+(H*(s11-s22)**2)+(2*N*s12**2))**0.5

    #Ratio of the equivalent stress to the yield stress to establish the stress threshhold (Nondimensionalized equivalent stress)
    S_y=123
    S_eqN=Sig_eq/S_y
   
    #Hydrostatic Stress
    S_h=(s11+s22)/3

    #Triaxiality
    Tri=S_h/Sig_eq

    #J3 invariant
    J3=-(1/27)*(s11+s22)*((s11-2*s22)*(s22-2*s11)-(9*s12**2))

    #Lode parameter
    v=(27/2)*(J3/Sig_eq**3)
    c=np.arccos(v)
    lode=1-((2/math.pi)*c)
    lodeR=np.round(lode)

    #Save new state variables with minimum condition applied in a new array
    state_v=np.vstack([s11,s22,s12,S1_S2R,Tri,lode,lodeR,S_eqN])
    state_var=np.transpose(state_v)

    #calculate the length of the state variable matrix
    N=len(s11)
    #print(N)

    # Initialize counts for rows with 1, 2, and 3 elements
    tension_count1 = 0
    tension_count2 = 0
    compression_count1 = 0
    compression_count2 = 0
    shear_count = 0
    biaxial_tension_count=0
    biaxial_comp_count=0

    BT=2/3
    TI=0.5
    T=1/3
    SI=1/6
    S=0
    CI=-1/6
    C=-1/3
    BI=-0.5
    BC=-2/3

    p=1

    # Iterate through each row in the matrix
    for i in range(N):
        if state_var[i,4] == T and state_var[i,7]>=p and state_var[i,0]>state_var[i,1]:
            tension_count1+=1
        elif state_var[i,4] == T and state_var[i,7]>=p and state_var[i,1]>state_var[i,0]:
            tension_count2+=1
        elif SI <= state_var[i, 4] <= TI and state_var[i,5]> 0 and state_var[i,7]>=p and state_var[i,0]>state_var[i,1]:
            tension_count1+=1
        elif SI <= state_var[i, 4] <= TI and state_var[i,5]> 0 and state_var[i,7]>=p and state_var[i,1]>state_var[i,0]:
            tension_count2+=1
        elif state_var[i, 4] >= TI and state_var[i,5]> 0 and state_var[i,7]>=p and state_var[i,0]>state_var[i,1]:
            tension_count1+=1
        elif state_var[i, 4] >= TI and state_var[i,5]> 0 and state_var[i,7]>=p and state_var[i,1]>state_var[i,0]:
            tension_count2+=1
        elif state_var[i,4] ==S and state_var[i,7]>=p:
            shear_count +=1
        elif CI <= state_var[i,4] <= SI and state_var[i,6] ==0 and state_var[i,7]>=p:
            shear_count +=1
        elif state_var[i,4] == C and state_var[i,7]>=p and np.abs(state_var[i,0])>np.abs(state_var[i,1]):
            tension_count1+=1
        elif state_var[i,4] == C and state_var[i,7]>=p and np.abs(state_var[i,1])>np.abs(state_var[i,0]):
            tension_count2+=1
        elif BI <= state_var[i, 4] <= CI and state_var[i,5]< 0 and state_var[i,7]>=p and np.abs(state_var[i,0])>np.abs(state_var[i,1]):
            tension_count1+=1
        elif BI <= state_var[i, 4] <= CI and state_var[i,5]< 0 and state_var[i,7]>=p and np.abs(state_var[i,1])>np.abs(state_var[i,0]):
            tension_count2+=1
        elif state_var[i, 4] <= BI and state_var[i,5]< 0 and state_var[i,7]>=p and np.abs(state_var[i,0])>np.abs(state_var[i,1]):
            tension_count1+=1 
        elif state_var[i, 4] <= BI and state_var[i,5]< 0 and state_var[i,7]>=p and np.abs(state_var[i,1])>np.abs(state_var[i,0]):
            tension_count2+=1
        elif state_var[i, 4] == BC and state_var[i,5]> 0 and state_var[i,7]>=p:
            biaxial_tension_count+=1
        elif state_var[i, 4] <= BI and state_var[i,5]> 0 and state_var[i,7]>=p:
            biaxial_tension_count+=1
        elif state_var[i, 4] == BT and state_var[i,5]< 0 and state_var[i,7]>=p:
            biaxial_tension_count+=1
        elif state_var[i, 4] >= TI and state_var[i,5]< 0 and state_var[i,7]>=p:
            biaxial_tension_count+=1

    Totalcount=tension_count2+shear_count+tension_count1
    
    if Totalcount==0:
        return np.nan, np.nan
    else:

        P_tension1_3=(tension_count1)/Totalcount
        P_tension2_3=(tension_count2)/Totalcount
        P_shear_3=(shear_count)/Totalcount


        #Information Entropy
        #ENTROPY in the 11 direction
        if P_tension1_3==0:
            E_tension1_3=0
        else :
            E_tension1_3=-P_tension1_3*math.log(P_tension1_3)

        #ENTROPY in the 22 direction
        if P_tension2_3==0:
            E_tension2_3=0
        else :
            E_tension2_3=-P_tension2_3*math.log(P_tension2_3)

        #ENTROPY in the 12 direction
        if P_shear_3==0:
            E_shear_3=0
        else :
            E_shear_3=-P_shear_3*math.log(P_shear_3)
        
    #Joint entropy of all state variables

        Entropy=E_tension2_3+E_shear_3+E_tension1_3
            

        return Entropy

In [ ]:
#Objective function for optimization
def objective(trial):
    import numpy as np
    import os, time

    # Define constant values for eigenvalues 11-25 (you can adjust these as needed)
    constant_eigenvalues = [0.1] * 15  
    
    # Define first 10 eigenvalues as optimization variables (bounds 0–100)
    tunable_eigenvalues = [
        trial.suggest_uniform(f"L{i}", 0, 100) for i in range(1, 11)
    ]
    
    # Combine tunable and constant eigenvalues
    eigenvalues = tunable_eigenvalues + constant_eigenvalues

    # Create KL specimen
    abaqus_script = create_kl_specimen(
        eigenvalues=eigenvalues,
        target_void_fraction=0.25,
        corner_rounding=True,
        min_radius=0.5
    )

    # Run the Abaqus job (uncomment if running in supported environment)
    !abaqus cae noGUI=kl_specimen_abaqus.py

    # Load stress data
    data_1 = np.loadtxt('stress.txt', dtype="float", delimiter=',')
    Entropy= cal_entropy(data_1)

    # --- Logging results ---
    if not os.path.exists('optimization_results'):
        os.makedirs('optimization_results')

    results_file = 'optimization_results/optimization_log.csv'
    write_header = not os.path.exists(results_file)

    with open(results_file, 'a') as f:
        if write_header:
            header = "Trial,Timestamp," + ",".join([f"L{i}" for i in range(1, 26)]) + ",Entropy\n"
            f.write(header)

        timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
        param_values = ",".join([f"{val:.6f}" for val in eigenvalues])
        f.write(f"{trial.number},{timestamp},{param_values},{Entropy:.6f}\n")

    # Save trial details in a separate file
    trial_file = f'optimization_results/trial_{trial.number:04d}.txt'
    with open(trial_file, 'w') as f:
        f.write(f"Trial Number: {trial.number}\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write("Parameters (Tunable L1-L10):\n")
        for i, val in enumerate(tunable_eigenvalues, start=1):
            f.write(f"  L{i}: {val:.6f}\n")
        f.write("Parameters (Constant L11-L25):\n")
        for i, val in enumerate(constant_eigenvalues, start=11):
            f.write(f"  L{i}: {val:.6f} (constant)\n")
        f.write(f"Entropy Result: {Entropy:.6f}\n")


    # Return objective(s)
    return Entropy

In [ ]:
# Bayesian optimization to maximize entropy
np.random.seed(0)
# Create a study object for multiobjective optimization
study = optuna.create_study(
    direction=['maximize']  # Specify the directions for the objectives
    sampler=TPESampler(seed=0)  # Use the TPESampler and set the random seed
)

# Optimize the study
study.optimize(objective, n_trials=1000)